In [1]:
# This accelerator provides a function to retrieve a secret value from AWS secrets manager. The secret name along with the region must be passed into the function as parameters. The code has been adapted into a parameterized function from the canned template provided by AWS.
import json, boto3, time, requests, io, base64
import pandas as pd
from botocore.exceptions import ClientError

def get_secret(secret_name, region_name):

    #secret_name = "AmazonSageMaker-gmaps"
    #region_name = "us-east-2"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    # In this sample we only handle the specific exceptions for the 'GetSecretValue' API.
    # See https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
    # We rethrow the exception by default.

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException':
            # Secrets Manager can't decrypt the protected secret text using the provided KMS key.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException':
            # An error occurred on the server side.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException':
            # You provided an invalid value for a parameter.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException':
            # You provided a parameter value that is not valid for the current state of the resource.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException':
            # We can't find the resource that you asked for.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
    else:
        # Decrypts secret using the associated KMS CMK.
        # Depending on whether the secret is a string or binary, one of these fields will be populated.
        if 'SecretString' in get_secret_value_response:
            secret = get_secret_value_response['SecretString']
            #secret = json.loads(secret)
            return json.loads(secret)
        else:
            decoded_binary_secret = base64.b64decode(get_secret_value_response['SecretBinary'])
            return json.loads(secret)

In [2]:
# This accelerator provides a standardized pattern for retrieving Athena query results based on the execution id.
# This code is adapted from Evan Perotti from http://securityriskadvisors.com/blog/creating-a-project-sonar-fdns-api-with-aws/ and was adapted from the Lambda.

def retrieveresults(execid):
    athena = boto3.client('athena')
    s3 = boto3.client('s3')
    queryres = athena.get_query_execution(
        QueryExecutionId = execid
    )
    
    # Athena query checking code is from https://medium.com/dataseries/automating-athena-queries-from-s3-with-python-and-save-it-as-csv-8917258b1045
    # Loop until results are ready or fail after 5 minutes
    status = 'RUNNING'
    iterations = 60
    
    while (iterations>0):
        iterations = iterations - 1
        response_get_query_details = athena.get_query_execution(
        QueryExecutionId = execid
        )
        status = response_get_query_details['QueryExecution']['Status']['State']
        print(status)
        if (status == 'FAILED') or (status == 'CANCELLED'):
            return False, False
        elif status == 'SUCCEEDED':
            try:
                outputloc = queryres['QueryExecution']['ResultConfiguration']['OutputLocation']
                full = outputloc[5:] # trim s3:// prefix
                bucketloc = full.split('/')[0] # get bucket from full path
                keyloc = full.replace(bucketloc,'')[1:] # get key and remove starting /
    
                url = s3.generate_presigned_url(
                    'get_object',
                    Params={
                    'Bucket':bucketloc,
                    'Key':keyloc
                    }
                )
                return url
            except:
                url = "No results"
                return url
        else:
            time.sleep(5)

In [3]:
# This accelerator provides a standardized function for passing queries to Athena.

def queryathena(athenadb, athenabucket, query):
    athena = boto3.client('athena')
    qexec = athena.start_query_execution(
        QueryString=query,
        QueryExecutionContext={
            'Database':athenadb
        },
        ResultConfiguration={
            'OutputLocation':athenabucket
        }
    )
    execid = qexec['QueryExecutionId']
    return execid

In [2]:
# Prepare GoogleMaps API integration

# pip install gmaps
#https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html
import gmaps
import gmaps.datasets

def prepare_location(df_min):
    #pandas chaining
    #1 - group by lat/long
    #2 - get the size of a column to consolidate groupby object
    #3 - take it from a indexed series to a dataframe
    #4 - reset the index so that groupby is flattened and can be referenced
    #5 - rename the size column (0) to 'count'
    df_plot = df_min.groupby(['latitude', 'longitude']).size().to_frame().reset_index().rename(columns={0:'count'})
    return df_plot

# Need to enable extensions and widgets before figure will display
#jupyter nbextension enable --py gmaps
#jupyter nbextension enable --py widgetsnbextension
#Restart Jupyter

def get_heatmap(df_plot):
    locations = df_plot[['latitude', 'longitude']]
    weights = df_plot['count']
    secretname = 'AmazonSageMaker-gmaps'
    region_name = 'us-east-1'
    api = get_secret(secretname, region_name)
    apisecret = api['gmapsapi']
    gmaps.configure(api_key=apisecret)
    fig = gmaps.figure()
    #fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
    fig.add_layer(gmaps.heatmap_layer(locations))
    return fig

In [5]:
# Dependency to have 'pip install geoip' installed from configuration notebook.
import geoip2.database
reader = geoip2.database.Reader('tools/GeoLite2-City.mmdb')

#create a function to find the lat/long of the addy based on maxmind reader
def lookup_location(IPAddress):
    try:
        response = reader.city(IPAddress)
        # TO-DO - investigate return variables. May be a mismatch with state and city.
        return round(response.location.latitude,3), round(response.location.longitude,3), response.country.name, response.city.name
    except:
        # If private IP address, return address at center of map
        return 0,0,0,0
        #return null

#function to get a dual output into different columns in a dataframe
#https://stackoverflow.com/questions/23690284/pandas-apply-function-that-returns-multiple-values-to-rows-in-pandas-dataframe
def apply_and_concat(dataframe, field, func, column_names):
    return pd.concat((
                         dataframe,
                         dataframe[field].apply(
                         lambda cell: pd.Series(func(cell), index=column_names))), axis=1)

def get_location(df_ip,columnIP):
    df_min = apply_and_concat(df_ip, columnIP, lookup_location, ['latitude', 'longitude', 'country', 'locality'])
    # Remove the rows with missing lat/long
    df_min = df_min[(df_min.latitude != 0.000)&(df_min.longitude != 0.000)]
    return df_min

In [2]:
# Configure SNS topic and text messaging
def notify_user(phonenumber,message):
    import boto3
    client = boto3.client('sns')
    client.publish(PhoneNumber=phonenumber, Message=message)

# Example function call syntax
# notify_user('+1XXXXXXXXXX', 'Results are done')

In [ ]:
# upload files to S3
def upload_file(file_name, bucket, object_name=None):
    
    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True